In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np

# Size of our encoded representations (compression factor)
encoding_dim = 3  # 3 floats -> compression of factor 2, assuming the input is 6 floats

# This is our input placeholder for a one-hot encoded vector with 6 bits
input_img = Input(shape=(6,))

# "encoded" is the encoded representation of the input
encoded = Dense(16, activation='relu')(input_img)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(8, activation='relu')(encoded)
decoded = Dense(16, activation='relu')(decoded)
decoded = Dense(6, activation='softmax')(decoded)

# This model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# Create a placeholder for an encoded (3-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layers = autoencoder.layers[-3:]
# Create the decoder model
decoder = Model(encoded_input, decoder_layers[2](decoder_layers[1](decoder_layers[0](encoded_input))))

# Compile the model specifying the optimizer and loss function for training
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')

# Summary of the model
autoencoder.summary()


# Let's create some data to encode and decode
# We'll create a set of 6-bit one-hot encoded vectors
data_size = 1000
one_hot_vectors = np.eye(6)[np.random.choice(6, data_size)]

# Train the autoencoder
autoencoder.fit(one_hot_vectors, one_hot_vectors,
                epochs=500,
                batch_size=256,
                shuffle=True)

# Now let's test the decoder
# Encode and decode some digits
encoded_imgs = encoder.predict(one_hot_vectors)
decoded_imgs = decoder.predict(encoded_imgs)

# Let's check the results of the encoding and decoding for the first example
print("Original:", one_hot_vectors[0])
print("Encoded:", encoded_imgs[0])
print("Decoded:", decoded_imgs[0])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 6)]               0         
                                                                 
 dense_6 (Dense)             (None, 16)                112       
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 3)                 27        
                                                                 
 dense_9 (Dense)             (None, 8)                 32        
                                                                 
 dense_10 (Dense)            (None, 16)                144       
                                                                 
 dense_11 (Dense)            (None, 6)                 102 